In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
data = pd.read_excel('New_Bank_loan_data.xlsx') 

In [3]:
# Handle missing values by dropping rows with NaN values
data.dropna(inplace=True)

Now we are Performing EDA for checking the data structures

In [4]:
data.head()

,ID,Age,Gender,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Home Ownership,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,M,1,49.0,91107,4,1.6,1,0,Home Owner,0,1,0,0.0,0
1,2,45,M,19,34.0,90089,3,1.5,1,0,Rent,0,1,0,0.0,0
2,3,39,M,15,11.0,94720,1,1.0,1,0,Rent,0,0,0,0.0,0
3,4,35,M,9,100.0,94112,1,2.7,2,0,Rent,0,0,0,0.0,0
4,5,35,M,8,45.0,91330,4,1.0,2,0,Rent,0,0,0,0.0,1


In [5]:
data.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
count,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000
mean,1895.815145,46.527521,20.186446,75.709831,93139.557111,2.388482,2.002471,1.881005,59.804963,0.107541,0.062361,0.592428,0.289532
std,1071.987943,30.827425,11.455234,47.258473,2296.958046,1.145476,1.805444,0.836298,105.736156,0.309849,0.241848,0.491461,0.453617
min,1.000000,2.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1008.000000,35.000000,10.000000,39.000000,92007.500000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1891.000000,46.000000,20.000000,65.000000,93407.000000,2.000000,1.600000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,2747.500000,55.000000,30.000000,104.000000,94607.500000,3.000000,2.600000,3.000000,104.000000,0.000000,0.000000,1.000000,1.000000
max,4981.000000,978.000000,43.000000,205.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 0 to 4980
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  3143 non-null   int64  
 1   Age                 3143 non-null   int64  
 2   Gender              3143 non-null   object 
 3   Experience          3143 non-null   int64  
 4   Income              3143 non-null   float64
 5   ZIP Code            3143 non-null   int64  
 6   Family              3143 non-null   int64  
 7   CCAvg               3143 non-null   float64
 8   Education           3143 non-null   int64  
 9   Mortgage            3143 non-null   int64  
 10  Home Ownership      3143 non-null   object 
 11  Personal Loan       3143 non-null   object 
 12  Securities Account  3143 non-null   int64  
 13  CD Account          3143 non-null   int64  
 14  Online              3143 non-null   float64
 15  CreditCard          3143 non-null   int64  
dtypes: flo

In [7]:
data.isnull().values.any()

False

In [8]:
data.size

50288

In [9]:
# Extract target variable
y = data['Personal Loan'].astype(str)
data.drop(columns=['Personal Loan'], inplace=True)

In [10]:
# Identifying the problems 
# X = data.drop('Personal Loan', axis=1)
# y = data['Personal Loan']
# problematic_cols = []
# for col in X.select_dtypes(include=['object']).columns:
#     try:
#         label_encoder = LabelEncoder()
#         X[col] = label_encoder.fit_transform(X[col])
#     except:
#         problematic_cols.append(col)
#         print(f"Column '{col}' caused an error.")

# print("Problematic Columns:", problematic_cols)

In [11]:
# Split the data into train and eval sets
X_train, X_eval, y_train, y_eval = train_test_split(data, y, test_size=0.20, random_state=0)

In [12]:
# Preprocess categorical columns (one-hot encoding)
categorical_columns = ['Gender', 'Home Ownership']
X_train = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
X_eval = pd.get_dummies(X_eval, columns=categorical_columns, drop_first=True)

In [13]:
# Add columns for missing values in evaluation set
for col in X_train.columns:
    if col not in X_eval.columns:
        X_eval[col] = 0

# Reorder evaluation columns to match training columns
X_eval = X_eval[X_train.columns]

In [14]:
# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_eval_scaled = scaler.fit_transform(X_eval)

In [15]:
# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
precdicted_classes = model.predict(X_train_scaled)

In [16]:
# Make predictions on eval data
y_pred = model.predict(X_eval_scaled)
accuracy = accuracy_score(y_eval, y_pred)
parameters = model.coef_


In [17]:
# Calculate accuracy
print("Accuracy:", accuracy)
print(parameters)

Accuracy: 0.9618441971383148
[[ 0.69066583  0.18452385  0.10029251  2.41442821 -0.11019572  0.73133538
   0.3310511   1.39433186 -0.0088593  -0.42059917  1.01344306 -0.19678616
  -0.53915875 -0.23008314 -0.14645106  0.19308167  0.13482352  0.13473525
   0.30224633]]
